In [1]:
import pandas as pd
import numpy as np
from io import StringIO


Imagina que eres un investigador en el mundo de Pokémon y has recibido una base de datos con información desactualizada y desordenada sobre avistamientos de Pokémon. Tu misión es limpiar y actualizar esta base de datos para que pueda ser utilizada en un estudio sobre la población de Pokémon en la región.

Datos Iniciales

Los datos iniciales contienen las siguientes columnas:

    SightingDate: Fecha del avistamiento.
    TrainerID: Identificación del entrenador que reportó el avistamiento.
    PokemonName: Nombre del Pokémon avistado.
    CP: Puntos de combate del Pokémon reportado.
    HP: Puntos de salud del Pokémon reportado.
    Type: Tipo del Pokémon.
    Weather: Clima durante el avistamiento.

# 1 Carga de Datos

In [14]:

# path_datos = 'pokemon.csv'
# TODO completa el codigo para cargar los datos
# df = pd.read_csv(path_datos)

# Especifica la ruta al archivo CSV y la codificación ISO
path_datos = 'pokemon.csv'
codificacion_iso = 'iso-8859-1'  # Asegúrate de usar la codificación correcta

# Carga los datos desde el archivo CSV
df = pd.read_csv(path_datos, encoding=codificacion_iso)
print(df)

               SightingDate    SightingTimeUTC TrainerID PokémonName       CP  \
0  2023-11-08T14:00:00+0000  2023-11-08T14:00Z     TR123     Pikachu      500   
1  2023-07-12T09:30:00+0100  2023-07-12T08:30Z     TR456       Eevee  MISSING   
2  2023/02/23T22:15:00+0900  2023-02-23T13:15Z     TR789    Magicarp    1,000   
3  2023-04-30T06:45:00-0400  2023-04-30T10:45Z     TR101      Gengar      800   
4  2023-08-15T16:00:00+1000                NaN     TR102   Bulbasaur      750   

   HP          Type        Weather  
0  35      Electric          Clear  
1  55        Normal         Cloudy  
2  10         Water           Rain  
3  45         Ghost  PARTLY_CLOUDY  
4  50  Grass/Poison          Sunny  


# Limpieza de Datos

## 1 Normalizacion de Zonas Horarias

Normaliza la columna `'SightingTimeUTC'` a la zona horaria UTC y convierte `'SightingDate'` al mismo formato de tiempo.


In [21]:
import pytz

df['SightingDate'] = pd.to_datetime(df['SightingDate'])
print(df['SightingDate'])

0   2023-11-08 14:00:00+00:00
1   2023-07-12 08:30:00+00:00
2   2023-02-23 13:15:00+00:00
3   2023-04-30 10:45:00+00:00
4   2023-08-15 06:00:00+00:00
Name: SightingDate, dtype: datetime64[ns, UTC]


Compara si la fecha de la columna `'SightingDate'` coincide con la fecha en `'SightingTimeUTC'` una vez normalizada.


In [22]:
df['SightingDate'] = pd.to_datetime(df['SightingDate'], utc=True)
df['SightingTimeUTC'] = pd.to_datetime(df['SightingTimeUTC'], utc=True)
df['Compare'] = df['SightingDate'].dt.floor('T') == df['SightingTimeUTC'].dt.floor('T')

print(df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Compare  
0      500  35      Electric          Clear     True  
1  MISSING  55        Normal         Cloudy     True  
2    1,000  10         Water           Rain     True  
3      800  45         Ghost  PARTLY_CLOUDY     True  
4      750  50  Grass/Poison          Sunny    False  


Ajusta `'SightingTimeUTC'` a la zona horaria local de cada entrenador y crea una columna `'SightingTimeLocal'`.

In [24]:
df['SightingTimeLocal'] = df['SightingTimeUTC'].copy()
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Compare         SightingTimeLocal  
0      500  35      Electric          Clear     True 2023-11-08 14:00:00+00:00  
1  MISSING  55        Normal         Cloudy     True 2023-07-12 08:30:00+00:00  
2    1,000  10         Water           Rain     True 2023-02-23 13:15:00+00:00  
3      800  45         Ghost  PARTLY_CLOUDY     True 2023-04-30 10:45:00+00:00  
4      750  50  Grass/Poison          Sunny    False                       NaT  


Calcula el tiempo transcurrido desde el momento del avistamiento hasta `'ahora'` (tu hora local) y crea una columna `'TimeSinceSighting'`.

In [26]:
import datetime

hora_actual_local = datetime.datetime.now(pytz.timezone('America/New_York'))
# Asegúrate de que 'SightingTimeLocal' está en formato datetime y tiene zonas horarias asignadas
df['SightingTimeLocal'] = pd.to_datetime(df['SightingTimeLocal'], utc=True)

# Calcula el tiempo transcurrido desde el avistamiento hasta ahora
df['TimeSinceSighting'] = (hora_actual_local - df['SightingTimeLocal']).dt.total_seconds() / 60  # En minutos

print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Compare  \
0      500  35      Electric          Clear     True   
1  MISSING  55        Normal         Cloudy     True   
2    1,000  10         Water           Rain     True   
3      800  45         Ghost  PARTLY_CLOUDY     True   
4      750  50  Grass/Poison          Sunny    False   

          SightingTimeLocal  TimeSinceSighting  
0 2023-11-08 14:00:00+00:00       43839.466617  
1 2023-07-12 08:30:00+00:00      215529.466617  
2 2023-02-23 13:15:00+00:00      415404.4

## 2. Limpeiza de IDs

Llena los valores faltantes en `'TrainerID'` con el ID `'UNKNOWN'`.


In [27]:
df['TrainerID'].fillna('UNKNOWN', inplace=True)
print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Compare  \
0      500  35      Electric          Clear     True   
1  MISSING  55        Normal         Cloudy     True   
2    1,000  10         Water           Rain     True   
3      800  45         Ghost  PARTLY_CLOUDY     True   
4      750  50  Grass/Poison          Sunny    False   

          SightingTimeLocal  TimeSinceSighting  
0 2023-11-08 14:00:00+00:00       43839.466617  
1 2023-07-12 08:30:00+00:00      215529.466617  
2 2023-02-23 13:15:00+00:00      415404.4

## 3. Corrección de Nombres de Pokémon

Asegúrate de que los nombres de Pokémon estén capitalizados correctamente.


In [28]:
df['PokemonName'] =  df['PokémonName'].str.capitalize()

print(df)


               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

        CP  HP          Type        Weather  Compare  \
0      500  35      Electric          Clear     True   
1  MISSING  55        Normal         Cloudy     True   
2    1,000  10         Water           Rain     True   
3      800  45         Ghost  PARTLY_CLOUDY     True   
4      750  50  Grass/Poison          Sunny    False   

          SightingTimeLocal  TimeSinceSighting PokemonName  
0 2023-11-08 14:00:00+00:00       43839.466617     Pikachu  
1 2023-07-12 08:30:00+00:00      215529.466617       Eevee  
2 202

## 4. Conversión de 'CP' y 'HP' a Numéricos

Convierte `'CP'` y `'HP'` a valores numéricos, manejando los `'MISSING'` y comas como separadores de miles.

In [30]:
#Convierte el string 'MISSING' a NaN en la columna 'CP','HP':
df['CP'] = df['CP'].replace('MISSING', np.nan)
#Convierte las comas en separadores de miles en la columna 'CP':
df['CP'] = df['CP'].str.replace(',', '')
#Convierte en numérico
df['CP'] = pd.to_numeric(df['CP'], errors='coerce')


## 5. Estandarización de 'Type'

Divide la columna `'Type'` en `'PrimaryType'` y `'SecondaryType'` cuando hay dos tipos.


In [31]:
df[['PrimaryType', 'SecondaryType']] = df['Type'].str.split('/', n=1, expand=True)
print(df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

       CP  HP          Type        Weather  Compare         SightingTimeLocal  \
0   500.0  35      Electric          Clear     True 2023-11-08 14:00:00+00:00   
1     NaN  55        Normal         Cloudy     True 2023-07-12 08:30:00+00:00   
2  1000.0  10         Water           Rain     True 2023-02-23 13:15:00+00:00   
3   800.0  45         Ghost  PARTLY_CLOUDY     True 2023-04-30 10:45:00+00:00   
4   750.0  50  Grass/Poison          Sunny    False                       NaT   

   TimeSinceSighting PokemonName Prima

## 6. Corrección del Clima

Estándariza la columna `'Weather'` para que todos los valores sean mayúsculas.


In [32]:
df['Weather'] = df['Weather'].str.upper()
print(df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

       CP  HP          Type        Weather  Compare         SightingTimeLocal  \
0   500.0  35      Electric          CLEAR     True 2023-11-08 14:00:00+00:00   
1     NaN  55        Normal         CLOUDY     True 2023-07-12 08:30:00+00:00   
2  1000.0  10         Water           RAIN     True 2023-02-23 13:15:00+00:00   
3   800.0  45         Ghost  PARTLY_CLOUDY     True 2023-04-30 10:45:00+00:00   
4   750.0  50  Grass/Poison          SUNNY    False                       NaT   

   TimeSinceSighting PokemonName Prima

# 3 Analisis de Datos

## Agrupaciones

Agrupa el DataFrame por `'Type'` y calcula la suma de `'CP'` para cada grupo.

In [33]:
sum_cp_by_type = df.groupby('Type')['CP'].sum()
print(sum_cp_by_type)

Type
Electric         500.0
Ghost            800.0
Grass/Poison     750.0
Normal             0.0
Water           1000.0
Name: CP, dtype: float64


Después de la suma, agrega una columna que calcule la media de `'HP'` por cada `'Type'`, pero solo para aquellos Pokémon cuyo `'CP'` sea mayor que el promedio de `'CP'` de todo el DataFrame.

In [39]:
mean = df['CP'].mean()
#Filtramos a los pokemons con CP mayor a la media
filt = df[df['CP'] > mean]
#Agrupamos por tipo en el df filtrado
grupo = filt.groupby('Type')
#Obtenemos el promedio de HP del grupo filtrado
HPmean = grupo['HP'].mean()
#Creamos la nueva columna HPmean, mapeando los elementos de HPmean con los de type
df['HPmean'] = df['Type'].map(HPmean)
print(df)

               SightingDate           SightingTimeUTC TrainerID PokémonName  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00     TR123     Pikachu   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00     TR456       Eevee   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00     TR789    Magicarp   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00     TR101      Gengar   
4 2023-08-15 06:00:00+00:00                       NaT     TR102   Bulbasaur   

       CP  HP          Type        Weather  Compare         SightingTimeLocal  \
0   500.0  35      Electric          CLEAR     True 2023-11-08 14:00:00+00:00   
1     NaN  55        Normal         CLOUDY     True 2023-07-12 08:30:00+00:00   
2  1000.0  10         Water           RAIN     True 2023-02-23 13:15:00+00:00   
3   800.0  45         Ghost  PARTLY_CLOUDY     True 2023-04-30 10:45:00+00:00   
4   750.0  50  Grass/Poison          SUNNY    False                       NaT   

   TimeSinceSighting PokemonName Prima